### Minimale Drawing App (mit Erweiterungen aus Aufgabe)

Mit der Maus soll eine Linie gezeichnet werden können, vom Punkt A, wo die
linke Maustaste gedrückt wurde, bis zum Punkt B, wo die linke Maustaste losgelassen wird.

**Tastenbefehle**:  
1. Falls Maustaste nicht gedrückt:
   - c: clear
   - 1-9: Linienbreite setzen
   - Kleinbuchstabe: Stiftfarbe setzen
   - Grossbuchstabe: Füllfarbe setzen

<br>  

2. Falls Maustaste gedrückt:  
   Spezifiziere, wie A und B verbunden werden.
   - ESC: Abbruch (beim Loslassen der Maustaste wird nichts gezeichnet)
   - r/R: stroke/fill Rechteck, A und B sind gegenüberliegende Ecken 
   - c/C: stroke/fill Kreis mit Zentrum A durch B

### Implementation
Der Dictionary `colors` speichert die Farben und der
Dictionary `state` speichert die Position, wo die Maustaste gedrückt wurde und
welche Taste zuletzt gedrückt wurde. 

Gezeichnet wird auf den Hintergrund. Auf dem Vordergrund werden Hilfsinformationen angezeigt, wie z.B. 
Füllfarbe und Linienbreite.

- Wird die Maustaste gedrückt, wird die Mausposition `(x, y)` in
`state['mouse_down']` gespeichert.
Zudem wird ein Punkt in der Füllfarbe mit Radius linewidth/2 auf den Vordergrund gezeichnet, umrandet von einem Kreis in der Stiftfarbe.
- Wird eine Taste gedrückt wird diese in `state['key_pressed']` gespeichert,
  falls die Maustaste gedrückt ist, sonst wird der entsprechende Befehl ausgeführt.
- Wird die Maustaste losgelassen, so
  wird abhängig von der gespeicherten Taste etwas gezeichnet. Dann werden
  `state['mouse_down']` und `state['key_pressed']` auf `None` gesetzt. Der Vordergrund wird gelöscht.

In [ ]:
from ipywidgets import Output, Text


state = {'mouse_down': None,  # Position A (wo Maustaste gedrueckt)
         'key_pressed': None,  # letzte Taste, die gedrueckt waehrend Maustaste gedrueckt
         }

colors = {'r': 'red',  'o': 'orange', 'b': 'blue', 'g': 'green',
          'y': 'yellow', 'k': 'black', 'p': 'pink', 't': 'teal'}


out = Output(layout={'border': '1px solid black'})
eingabefeld = Text()


def set_color(key):
    color = colors[key.lower()]
    if key.isupper():
        fg.fill_style = color
        bg.fill_style = color
        print(f'fill-color set to {color}')
    else:
        fg.stroke_style = color
        bg.stroke_style = color
        print(f'stroke-color set to {color}')


def get_rect_and_circ(x, y):
    x0, y0 = state['mouse_down']
    width = x - x0
    height = y - y0
    radius = (width**2+height**2)**.5
    return x0, y0, width, height, radius


@out.capture(clear_output=True)
def on_mouse_down(x, y):
    state['mouse_down'] = (x, y)
    fg.fill_circle(x, y, radius=bg.line_width/2)
    fg.stroke_circle(x, y, radius=bg.line_width/2 + 3)
    print(f'A (state[\'mouse_down\']) set to ({int(x)}, {int(y)})')


@out.capture()
def on_mouse_up(x, y):
    fg.clear()
    if state['mouse_down'] is None:
        return

    print(f'B is ({int(x)}, {int(y)})')
    x0, y0, width, height, radius = get_rect_and_circ(x, y)
    key = state['key_pressed']

    if key == 'Escape':
        pass  # macht nichts
    elif key == 'R':
        bg.fill_rect(x0, y0, width, height)
    elif key == 'r':
        bg.stroke_rect(x0, y0, width, height)
    elif key == 'C':
        bg.fill_circle(x0, y0, radius)
    elif key == 'c':
        bg.stroke_circle(x0, y0, radius)
    elif key == 'e':
        bg.clear_rect(x0, y0, width, height)
    elif key == 't' and eingabefeld.value:
        bg.font = f'{abs(height)}px sans-serif'
        bg.fill_text(eingabefeld.value, x0+width/2, y0+height/2, max_width=abs(width))
        eingabefeld.value = ''
    else:
        bg.stroke_lines([(x0, y0), (x, y)])

    state['mouse_down'] = None
    state['key_pressed'] = None


@out.capture()
def on_mouse_move(x, y):
    if state['mouse_down'] is None:
        return

    fg.clear()
    x0, y0, width, height, radius = get_rect_and_circ(x, y)
    fg.stroke_rect(x0, y0, width, height)
    fg.stroke_circle(x0, y0, radius)
    fg.stroke_lines([(x0, y0), (x, y)])


@out.capture()
def on_mouse_out(x, y):
    fg.clear()
    state['mouse_down'] = None
    state['key_down'] = None
    print('values of state set to None')


@out.capture()
def on_key_down(key, *flags):
    if state['mouse_down']:
        state['key_pressed'] = key
        print(f'state[\'key_pressed\'] set to {key}')
        return

    if key == 'c':
        bg.clear()
    elif key.lower() in colors:
        set_color(key)
    elif key in '123456789':
        bg.line_width = int(key)
        print(f'linewidth set to {key}')

In [ ]:
from ipycanvas import MultiCanvas
from IPython.display import display

layout = {'border': '1px solid black'}
mcanvas = MultiCanvas(2, width=200, height=200, layout=layout)
bg, fg = mcanvas
bg.text_align = 'center'
bg.text_baseline = 'middle'

mcanvas.on_mouse_down(on_mouse_down)
mcanvas.on_mouse_up(on_mouse_up)
mcanvas.on_mouse_move(on_mouse_move)
mcanvas.on_mouse_out(on_mouse_out)
mcanvas.on_key_down(on_key_down)


display(mcanvas, eingabefeld, out)